In [ ]:
import os
if not os.path.exists('open.zip') and not os.path.exists('./data'): # Download data from google drive
    !pip install gdown
    !gdown https://drive.google.com/uc?id=13oGkm3Ao7fL2p51H62J68Gw630ABBR0g 
    !unzip open.zip -d data # unzip in the 'data' folder


In [ ]:
from prepare_origin import save_origin
if not os.path.exists('./data/origin'):  # make origin picture from train data
    save_origin('./data')